In [1]:
# main things we need to do:
# 0.⁠ ⁠prepare (collect data and clean) auth df - V
# 0.⁠ ⁠prepare (collect data and clean) events_df - V
# 0.⁠ ⁠preprocess auth df - fill null values
# 1.⁠ ⁠run matching (match_and_plot)
# 2.⁠ ⁠⁠check plots of different distance metrics results
# 3.⁠ ⁠⁠run diff_in_diff analysis for target authorities and each event (conviction etc.) (calc_diff_in_diff)
# 4.⁠ ⁠⁠check plots of DiD (compare_pred_true)
# 5.⁠ ⁠⁠check if we see the same results across different events (conviction etc.)
# 5.⁠ ⁠⁠check hyper params plot of DiD (plot_across_hyper_params)
# 6.⁠ ⁠⁠run diff_in_diff analysis for placebo auth - sample some from the control group, sample year for treatment and run the same piepline
# 7.⁠ ⁠⁠then - check that we dont see a major diff in diff there
# 8.⁠ ⁠⁠summarize results
# 9.⁠ ⁠⁠conclude in report

In [2]:
import pandas as pd
from pathlib import Path
src_path = Path.cwd().parents[0]

df_matching = pd.read_csv(src_path / 'data/matching_features_df.csv')
df_matching = df_matching.rename(columns={'authority_code': 'auth_id', 'authority_name': 'auth_name'})
df_matching

,year,auth_id,auth_name,is_treatment,district,distance_from_tel_aviv,year_of_municipal_status,council_members_number,socioeconomic_level,compactness_cluster,...,regular_budget_revenues,gov_participation,gov_participation_change,special_budget_revenues,property_tax_total_prev_year,property_tax_residential_prev_year,property_tax_business_prev_year,property_tax_total_current_year,property_tax_residential_current_year,property_tax_business_current_year
0,1999,26,ראש פינה,0,4,134.378618,1949,7.0,9.0,7.0,...,16192.0,4056.0,13.500,6813.0,9.150079e+04,19540.523333,2183.253333,41378.441674,8802.270211,2012.841105
1,2000,26,ראש פינה,0,4,134.378618,1949,7.0,7.0,7.0,...,15810.0,3366.0,-17.900,4927.0,9.150079e+04,19540.523333,2183.253333,41378.441674,8802.270211,2012.841105
2,2001,26,ראש פינה,0,4,134.378618,1949,7.0,7.0,7.0,...,16018.0,3569.0,4.900,5687.0,9.150079e+04,19540.523333,2183.253333,41378.441674,8802.270211,2012.841105
3,2002,26,ראש פינה,0,4,134.378618,1949,7.0,7.0,7.0,...,17902.0,3239.0,-2.700,3380.0,9.150079e+04,19540.523333,2183.253333,41378.441674,8802.270211,2012.841105
4,2003,26,ראש פינה,0,4,134.378618,1949,7.0,7.0,7.0,...,19165.0,2786.0,-28.700,6901.0,9.150079e+04,19540.523333,2183.253333,41378.441674,8802.270211,2012.841105
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2815,2018,9800,בנימינה-גבעת עדה*,0,0,44.388835,1950,13.0,8.0,10.0,...,97992.0,29670.0,1.200,30304.0,1.780200e+04,545.400000,44.400000,17647.500000,604.900000,49.600000
2816,2019,9800,בנימינה-גבעת עדה*,0,0,44.388835,1950,13.0,9.0,10.0,...,102963.0,30642.0,1.065,19963.0,3.816485e+06,71618.466667,3363.170000,17497.000000,614.000000,49.000000
2817,2020,9800,בנימינה-גבעת עדה,0,0,44.388835,1950,13.0,9.0,10.0,...,103935.0,36491.0,1.065,13207.0,3.816485e+06,71618.466667,3363.170000,840.000000,628.000000,53.000000
2818,2021,9800,בנימינה-גבעת עדה,0,0,44.388835,1950,13.0,8.0,10.0,...,111242.0,33234.0,1.065,25954.0,3.816485e+06,71618.466667,3363.170000,862.960000,641.120000,50.580000


In [13]:
df_treatment = df_matching[df_matching['is_treatment'] == 1]
df_control = df_matching[df_matching['is_treatment'] == 0]

In [14]:
df_control.groupby(['year'])['auth_id'].count()

year
1999    104
2000    104
2001    104
2002     60
2003    104
2004    104
2005    104
2006    104
2007    104
2008    104
2009    104
2010    104
2011    104
2012    104
2013    104
2014    104
2015    104
2016    104
2017    104
2018    104
2019    104
2020    104
2021    104
2022    104
Name: auth_id, dtype: int64

In [ ]:
df_treatment

In [ ]:
df_treatment.groupby(['year'])['auth_id'].count()

In [ ]:
treatment_ids = df_treatment['auth_id'].unique()
print(len(treatment_ids), treatment_ids)
control_ids = df_control['auth_id'].unique()
print(len(control_ids), control_ids)

In [ ]:
df_control

In [ ]:
from src.matching.create_vecs import create_vecs
control_vecs = create_vecs(df_control, method="agg")
# control_vecs = create_vecs(df_control, method="concat")

In [ ]:
from src.matching.matching_funcs import match_and_plot
results_dir = src_path / 'results'

k = 2
metric='euclidean' # 'euclidean' or 'cosine' or 'manhattan' or 'minkowski' or 'chebyshev' or 'hamming' or 'jaccard'
matching_cols=['feature1', 'feature2']
match_and_plot(df_treatment, df_control, matching_cols, k, metric)

In [3]:
infer_df = pd.read_csv(src_path / 'data/inference_features_df.csv')
infer_df = infer_df.rename(columns={'authority_code': 'auth_id'})
infer_df = infer_df.merge(df_matching[['auth_id', 'auth_name']], on='auth_id', how='left')
infer_df

,year,is_treatment,incident_year,incident_type,auth_id,total_in_migration,male_in_migration,female_in_migration,in_migration_age_15_29,total_out_migration,...,total_expenditures,expenditures_change,operations_expenditures,loan_repayment_expenditures,financing_expenditures,special_budget_expenditures,budget_surplus_deficit,annual_surplus_deficit,cumulative_deficit,auth_name
0,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.00,NaN,ראש פינה
1,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.00,NaN,ראש פינה
2,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.00,NaN,ראש פינה
3,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.00,NaN,ראש פינה
4,1999,0,NaN,NaN,26,127.0,67.0,60.0,41.0,87.0,...,22876.0,29.5,8470.0,NaN,129.0,NaN,261.0,261.00,NaN,ראש פינה
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
76346,2022,0,NaN,NaN,9800,617.0,331.0,286.0,84.0,536.0,...,130811.0,NaN,43052.0,4786.0,918.0,16634.0,50.0,0.05,-10.451,בנימינה-גבעת עדה*
76347,2022,0,NaN,NaN,9800,617.0,331.0,286.0,84.0,536.0,...,130811.0,NaN,43052.0,4786.0,918.0,16634.0,50.0,0.05,-10.451,בנימינה-גבעת עדה*
76348,2022,0,NaN,NaN,9800,617.0,331.0,286.0,84.0,536.0,...,130811.0,NaN,43052.0,4786.0,918.0,16634.0,50.0,0.05,-10.451,בנימינה-גבעת עדה
76349,2022,0,NaN,NaN,9800,617.0,331.0,286.0,84.0,536.0,...,130811.0,NaN,43052.0,4786.0,918.0,16634.0,50.0,0.05,-10.451,בנימינה-גבעת עדה


In [ ]:
infer_df.columns

In [4]:
infer_df[infer_df['is_treatment'] == 1]

,year,is_treatment,incident_year,incident_type,auth_id,total_in_migration,male_in_migration,female_in_migration,in_migration_age_15_29,total_out_migration,...,total_expenditures,expenditures_change,operations_expenditures,loan_repayment_expenditures,financing_expenditures,special_budget_expenditures,budget_surplus_deficit,annual_surplus_deficit,cumulative_deficit,auth_name
19878,1999,1,2012.0,arrest,1034,546.0,267.0,279.0,228.0,953.0,...,131211.0,-8.389252,45087.0,12581.0,2381.0,18593.0,NaN,NaN,NaN,קריית מלאכי
19879,1999,1,2012.0,arrest,1034,546.0,267.0,279.0,228.0,953.0,...,131211.0,-8.389252,45087.0,12581.0,2381.0,18593.0,NaN,NaN,NaN,קריית מלאכי
19880,1999,1,2012.0,arrest,1034,546.0,267.0,279.0,228.0,953.0,...,131211.0,-8.389252,45087.0,12581.0,2381.0,18593.0,NaN,NaN,NaN,קריית מלאכי
19881,1999,1,2012.0,arrest,1034,546.0,267.0,279.0,228.0,953.0,...,131211.0,-8.389252,45087.0,12581.0,2381.0,18593.0,NaN,NaN,NaN,קריית מלאכי
19882,1999,1,2012.0,arrest,1034,546.0,267.0,279.0,228.0,953.0,...,131211.0,-8.389252,45087.0,12581.0,2381.0,18593.0,NaN,NaN,NaN,קריית מלאכי
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
72549,2022,1,2008.0,conviction,9100,2713.0,1341.0,1372.0,536.0,2413.0,...,748306.0,NaN,168725.0,76567.0,1720.0,173439.0,466.0,-62.684,-42.571,נהרייה
72550,2022,1,2008.0,conviction,9100,2713.0,1341.0,1372.0,536.0,2413.0,...,748306.0,NaN,168725.0,76567.0,1720.0,173439.0,466.0,-62.684,-42.571,נהרייה
72551,2022,1,2008.0,conviction,9100,2713.0,1341.0,1372.0,536.0,2413.0,...,748306.0,NaN,168725.0,76567.0,1720.0,173439.0,466.0,-62.684,-42.571,נהרייה
72552,2022,1,2008.0,conviction,9100,2713.0,1341.0,1372.0,536.0,2413.0,...,748306.0,NaN,168725.0,76567.0,1720.0,173439.0,466.0,-62.684,-42.571,נהרייה


In [12]:
infer_df[infer_df['auth_id']==1034][['auth_name', 'year', 'incident_year', 'incident_type']].drop_duplicates()

,auth_name,year,incident_year,incident_type
19878,קריית מלאכי,1999,2012.0,arrest
19901,קריית מלאכי,1999,2013.0,conviction
19924,קריית מלאכי,2000,2012.0,arrest
19947,קריית מלאכי,2000,2013.0,conviction
19970,קריית מלאכי,2001,2012.0,arrest
19993,קריית מלאכי,2001,2013.0,conviction
20016,קריית מלאכי,2003,2012.0,arrest
20039,קריית מלאכי,2003,2013.0,conviction
20062,קריית מלאכי,2004,2012.0,arrest
20085,קריית מלאכי,2004,2013.0,conviction


In [10]:
infer_df[infer_df['is_treatment'] == 1].groupby('auth_id')['incident_type'].nunique()

auth_id
1034    2
1061    3
2630    1
2650    2
3000    3
5000    1
6200    3
6500    1
6700    3
6900    3
7000    3
7100    3
8000    1
8400    1
8600    2
9100    1
Name: incident_type, dtype: int64

In [15]:
columns_to_predict = [
    'total_in_migration', 'male_in_migration', 'female_in_migration',
    'in_migration_age_15_29', 'total_out_migration', 'male_out_migration',
    'female_out_migration', 'out_migration_age_15_29',
    'start_roads_length_km', 'start_roads_area_sqm', 'start_water_pipes_km',
    'start_sewage_pipes_km', 'start_drainage_pipes_km',
    'complete_roads_length_km', 'complete_roads_area_sqm',
    'complete_water_pipes_km', 'complete_sewage_pipes_km',
    'complete_drainage_pipes_km', 'satisfaction_municipal_services',
    'satisfaction_municipal_performance', 'satisfaction_residential_area',
    'satisfaction_cleanliness', 'satisfaction_parks_green_spaces',
    'feeling_safe_dark', 'contacting_local_authority', 'total_expenditures',
    'expenditures_change', 'operations_expenditures',
    'loan_repayment_expenditures', 'financing_expenditures',
    'special_budget_expenditures', 'budget_surplus_deficit',
    'annual_surplus_deficit', 'cumulative_deficit']

In [ ]:
from src.diff_in_diff.diff_in_diff_funcs import calc_diff_in_diff

calc_diff_in_diff(
    infer_df,
    columns_to_predict=columns_to_predict, 
    treatment_ids=df_treatment['auth_id'].tolist(),
    k=2,
    distance_metric='euclidean',
    diff_results_dir=(results_dir / 'diff'),
)